## CNN Classification on CIFAR-10

This notebook implements a complete Convolutional Neural Network (CNN) for classifying images from the CIFAR-10 dataset.  
The model takes 32x32 RGB images (center-cropped to 24x24) and processes them through a stack of convolutional, activation, and pooling layers before producing class probabilities.

### Architecture

- **Input:** 24x24x3

**Feature Extraction**
- **Conv1 (3x3, ReLU):** 24x24x64  
- **MaxPool (3x3, s=2):** 12x12x64
- **Conv2 (3x3, ReLU):** 12x12x32  
- **MaxPool (3x3, s=2):** 6x6x32
- **Conv3 (3x3, ReLU):** 6x6x20  
- **MaxPool (3x3, s=2):** 3x3x20

**Classifier**
- **Flatten:** 180 features  
- **FC:** 180 → 10  
- **Softmax:** class probabilities

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import time
import cv2
import re

### Functions Summary

- **load_pickle(file)**  
  Loads and returns a dictionary from a CIFAR-10 batch file using Python’s pickle module.

- **normalize_img(img)**  
  Computes the mean and standard deviation of a 24x24x3 image and returns a normalized version.

- **load_cnn_weights(filename)**  
  Parses a weight file, extracts tensors, reshapes them into their correct shapes, and returns all CNN parameters.

- **conv2d_relu(img, kernel, bias)**  
  Performs a 3x3 convolution (SAME padding), adds bias, applies ReLU, and returns output feature maps.
  
- **maxpool2d(img, pool_size=3, stride=2)**  
  Applies max pooling using a 3x3 window and stride 2.

- **flatten(img)**  
  Flattens a 3D feature map into a 1D vector.

- **dense_layer(input_vector, weight, bias)**  
  Computes a fully connected layer: \( W^T x + b \), returning logits.

- **softmax_activation(logits)**  
  Applies the softmax function to convert logits into class probabilities.

- **preprocess_cifar_image**  
  Reshapes a CIFAR-10 image (flat 3072-element array), converts from CHW to HWC, crops it from 32×32 to 24×24, then normalizes it.

- **forward_pass(img, tensors)**  
  Runs the complete CNN forward inference pass.

- **run_cnn_benchmark(tensors, kernel_label, data, n_samples, progress_every)**  
  Evaluates the model on a subset of the CIFAR-10 dataset, measuring:
  - overall accuracy
  - per-class accuracy
  - inference latency per image
  - total runtime
  - returns a statistics dictionary

- **print_benchmark_summary(result, class_names=None)**  
  Nicely formats and prints the benchmark results, including class-wise accuracy and ASCII bar-visualization.

- **get_predictions(n_samples)**  
  This function re-runs the network to create clean lists (y_true and y_pred) required for advanced matrix calculations.

- **calculate_confusion_matrix(y_true, y_pred, num_classes)**  
  Creates a 10x10 grid. Rows represent the actual object, and columns represent what the CNN predicted.

- **calculate_metrics(cnn)**  
  Calculate Precision, Recall, and F1 Score for each class based on cnn and implements the One-vs-All formulas provided in your prompt:
  - **Precision**: Of all images predicted as "Airplane", how many were actually airplanes?
  - **Recall**: Of all actual "Airplanes" in the dataset, how many did the model find?
  - **F1**: The harmonic balance between the two.The F1-score is often described as the harmonic mean of Precision and Recall. 

$$F1 = \frac{2 \times \text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}$$


In [ ]:
def load_pickle(file):
    """Load a CIFAR-10 batch file using pickle."""
    import pickle
    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data


def normalize_img(img):
    """Normalize a 24x24x3 image using mean and standard deviation."""
    mean = 0.0
    N = 24 * 24 * 3
    
    for i in range(24):
        for j in range(24):
            for c in range(3):
                mean += float(img[i][j][c])
    mean /= N

    std_dev = 0.0
    for i in range(24):
        for j in range(24):
            for c in range(3):
                std_dev += (float(img[i][j][c]) - mean)**2
    std_dev = np.sqrt(std_dev / N)

    print(f"Image mean: {mean}, std_dev: {std_dev}")
    return (img - mean) / max(std_dev, 1 / np.sqrt(N))


def load_cnn_weights(filename, kernel_size=3):
    """Parse CNN weight tensors from the text file and reshape them."""
    with open(filename, 'r') as f:
        content = f.read()

    tensors = {}
    sections = content.split('tensor_name:')[1:]

    for section in sections:
        lines = section.strip().split('\n', 1)
        tensor_name = lines[0].strip()

        if len(lines) > 1:
            array_text = lines[1]
            start_idx = array_text.find('[')
            end_idx = array_text.rfind(']')
            if start_idx != -1 and end_idx != -1:
                numbers_text = array_text[start_idx+1:end_idx]
                numbers = re.findall(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?', numbers_text)
                tensor_data = np.array([float(x) for x in numbers], dtype=np.float32)
                tensors[tensor_name] = tensor_data

    tensors['conv1/weights'] = tensors['conv1/weights'].reshape(kernel_size, kernel_size, 3, 64)
    tensors['conv2/weights'] = tensors['conv2/weights'].reshape(kernel_size, kernel_size, 64, 32)
    tensors['conv3/weights'] = tensors['conv3/weights'].reshape(kernel_size, kernel_size, 32, 20)
    tensors['local3/weights'] = tensors['local3/weights'].reshape(180, 10)

    return tensors


def conv2d_relu(img, kernel, bias):
    """
    Apply a convolution with SAME padding followed by ReLU.
    """
    H, W, C_in = img.shape
    kernel_size, _, _, C_out = kernel.shape
    pad_size = kernel_size // 2
    
    padded_img = np.pad(img,
                        pad_width=((pad_size, pad_size),
                                   (pad_size, pad_size),
                                   (0, 0)),
                        mode='constant',
                        constant_values=0)
    
    output = np.zeros((H, W, C_out), dtype=np.float32)
    
    for f in range(C_out):
        for i in range(kernel_size):
            for j in range(kernel_size):
                for c in range(C_in):
                    output[:, :, f] += padded_img[i:i+H, j:j+W, c] * kernel[i, j, c, f]
        
        output[:, :, f] = np.maximum(0, output[:, :, f] + bias[f])
    
    return output


def maxpool2d(img, pool_size=3, stride=2):
    """
    Apply 2D max pooling with a given window and stride.
    """
    H, W, C = img.shape
    out_h = int(np.ceil((H - pool_size) / stride)) + 1
    out_w = int(np.ceil((W - pool_size) / stride)) + 1
    
    output = np.zeros((out_h, out_w, C), dtype=np.float32)
    
    for c in range(C):
        for i in range(out_h):
            for j in range(out_w):
                h_start = i * stride
                w_start = j * stride
                output[i, j, c] = np.max(img[h_start:h_start+pool_size,
                                            w_start:w_start+pool_size, c])
    
    return output


def flatten(img):
    """Flatten a 3D array (HxWxC) into a 1D vector."""
    return img.flatten().astype(np.float32)


def dense_layer(x, weight, bias):
    """Fully connected layer: Wᵀx + b."""
    return np.dot(x, weight) + bias


def softmax_activation(logits):
    """Compute softmax probabilities."""
    shifted = logits - np.max(logits)
    exp_vals = np.exp(shifted)
    return exp_vals / np.sum(exp_vals)


def preprocess_cifar_image(flat_img):
    """Reshape, crop, and normalize a CIFAR-10 image."""
    img = flat_img.reshape(3, 32, 32).transpose(1, 2, 0)
    img = img[4:28, 4:28, :]
    return normalize_img(img)


def forward_pass(img, tensors):
    """Run the CNN forward pass and return probabilities."""
    o1 = conv2d_relu(img, tensors['conv1/weights'], tensors['conv1/biases'])
    p1 = maxpool2d(o1)
    o2 = conv2d_relu(p1, tensors['conv2/weights'], tensors['conv2/biases'])
    p2 = maxpool2d(o2)
    o3 = conv2d_relu(p2, tensors['conv3/weights'], tensors['conv3/biases'])
    p3 = maxpool2d(o3)
    vec = flatten(p3)
    logits = dense_layer(vec, tensors['local3/weights'], tensors['local3/biases'])
    return softmax_activation(logits)


def run_cnn_benchmark(tensors, kernel_label, data, n_samples=1000, progress_every=100):
    """Benchmark a kernel size on n_samples images and collect stats."""
    accuracy_count = 0
    class_correct = np.zeros(10, dtype=int)
    class_total = np.zeros(10, dtype=int)
    per_image_times = []
    
    print(f"\nTesting CNN ({kernel_label} kernel) on {n_samples} images...\n")
    start_time = time.time()

    for idx in range(n_samples):
        img = preprocess_cifar_image(data[b'data'][idx])
        t0 = time.time()
        probs = forward_pass(img, tensors)
        per_image_times.append(time.time() - t0)
        predicted_label = int(np.argmax(probs))
        true_label = int(data[b'labels'][idx])
        
        if predicted_label == true_label:
            accuracy_count += 1
            class_correct[true_label] += 1
        class_total[true_label] += 1
        
        if progress_every and (idx + 1) % progress_every == 0:
            elapsed = time.time() - start_time
            avg = elapsed / (idx + 1)
            eta = avg * (n_samples - idx - 1)
            print(f"[{idx+1:5d}/{n_samples}] Accuracy: {accuracy_count/(idx+1)*100:.2f}% | "
                  f"Time: {elapsed:.1f}s | ETA: {eta:.1f}s")
    
    elapsed_total = time.time() - start_time
    per_class_accuracy = np.zeros_like(class_correct, dtype=np.float32)
    non_zero_mask = class_total > 0
    per_class_accuracy[non_zero_mask] = class_correct[non_zero_mask] / class_total[non_zero_mask] * 100
    
    return {
        'kernel': kernel_label,
        'N': n_samples,
        'accuracy_count': accuracy_count,
        'overall_accuracy': accuracy_count / n_samples * 100,
        'class_correct': class_correct,
        'class_total': class_total,
        'per_class_accuracy': per_class_accuracy,
        'elapsed_total': elapsed_total,
        'avg_time_ms': elapsed_total / n_samples * 1000,
        'per_image_times_ms': np.array(per_image_times) * 1000
    }


def print_benchmark_summary(result, class_names=None):
    """Pretty-print the benchmark output for a kernel."""
    print(f"\n{'='*60}")
    print(f"FINAL RESULTS ({result['kernel']} kernel)")
    print(f"{'='*60}")
    print(f"Total images tested: {result['N']}")
    print(f"Correct predictions: {result['accuracy_count']}")
    print(f"Overall Accuracy: {result['overall_accuracy']:.2f}%")
    print(f"Total time: {result['elapsed_total']:.2f}s")
    print(f"Avg time per image: {result['avg_time_ms']:.2f} ms")
    
    print(f"\n{'='*60}")
    print("Per-Class Accuracy")
    print(f"{'='*60}")
    
    if class_names is None:
        class_names = [f'class_{i}' for i in range(len(result['per_class_accuracy']))]
    
    for idx, name in enumerate(class_names):
        total = result['class_total'][idx]
        acc = result['per_class_accuracy'][idx]
        bar = "█" * int(acc / 2) if total else ""
        print(f"{idx}. {name:12s}: {result['class_correct'][idx]:4d}/{total:4d} = {acc:6.2f}% {bar}")

def get_predictions(tensors, data, n_samples=100):
    """Run inference to collect lists of true and predicted labels."""
    y_true = []
    y_pred = []
    
    print(f"Collecting predictions for {n_samples} images...")
    
    for i in range(n_samples):
        img = preprocess_cifar_image(data[b'data'][i])
        probs = forward_pass(img, tensors)
        y_pred.append(int(np.argmax(probs)))
        y_true.append(int(data[b'labels'][i]))
        
    return np.array(y_true), np.array(y_pred)


def calculate_confusion_matrix(y_true, y_pred, num_classes=10):
    """
    Compute Confusion Matrix C where C[i, j] is the number of observations
    known to be in group i and predicted to be in group j.
    """
    cm = np.zeros((num_classes, num_classes), dtype=int)
    for t, p in zip(y_true, y_pred):
        cm[t][p] += 1
    return cm


def calculate_metrics(cm):
    """
    Calculate Precision, Recall, and F1 Score for each class based on CM.
    """
    num_classes = cm.shape[0]
    
    precisions = []
    recalls = []
    f1_scores = []
    
    for i in range(num_classes):
        TP = cm[i, i]
        FP = np.sum(cm[:, i]) - TP
        FN = np.sum(cm[i, :]) - TP
        TN = np.sum(cm) - (TP + FP + FN)
        
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)
        
    return precisions, recalls, f1_scores

Load files for test data and tensor weights

In [ ]:
test_data = load_pickle("../dataset/cifar-10-batches-py/test_batch")
tensors_3x3 = load_cnn_weights('../dataset/CNN_coeff_3x3.txt', kernel_size=3)
tensors_5x5 = load_cnn_weights('../dataset/CNN_coeff_5x5.txt', kernel_size=5)

In [ ]:
img = test_data[b'data'][0].reshape(3, 32, 32).transpose(1, 2, 0)[4:28, 4:28, :]
print(img[0][0])
normalize_img(img)

Visualize 9 images from data

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(16, 16))
for i in range(3):
    for j in range(3):
        idx = i * 3 + j
        img = test_data[b'data'][idx].reshape(3,32,32).transpose(1,2,0)
        img = img[4:28, 4:28, :]
        img = normalize_img(img)
        axes[i, j].imshow((img - img.min()) / (img.max() - img.min()))
        axes[i, j].axis('on')
        axes[i, j].set_title(f"Image {test_data[b'filenames'][idx].decode('utf-8')}")
plt.show()

CNN Accuracy Testing

In [ ]:
# CIFAR-10 class names
cifar10_classes = [
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'
]

N = 1000
kernel_configs = {
    '3x3': tensors_3x3,
    '5x5': tensors_5x5
}
benchmark_results = {}

for kernel_name, tensor_set in kernel_configs.items():
    result = run_cnn_benchmark(tensor_set, kernel_name, test_data, n_samples=N, progress_every=100)
    benchmark_results[kernel_name] = result
    print_benchmark_summary(result, cifar10_classes)

print(f"\n{'#'*60}")
print("Kernel Comparison Summary")
print(f"{'#'*60}")
print(f"{'Kernel':<10}{'Accuracy (%)':>15}{'Avg ms/img':>15}{'Total Time (s)':>18}")
print("-" * 60)
for kernel_name, result in benchmark_results.items():
    print(f"{kernel_name:<10}{result['overall_accuracy']:>15.2f}{result['avg_time_ms']:>15.2f}{result['elapsed_total']:>18.2f}")

best_kernel = max(benchmark_results.values(), key=lambda x: x['overall_accuracy'])
print("-" * 60)
print(f"Best accuracy achieved with the {best_kernel['kernel']} kernel ({best_kernel['overall_accuracy']:.2f}%).")

In [ ]:
if not benchmark_results:
    raise RuntimeError("Run the benchmark cell before visualizing results.")

ordered_kernels = [k for k in ['3x3', '5x5'] if k in benchmark_results]
if len(ordered_kernels) < len(benchmark_results):
    ordered_kernels.extend(k for k in benchmark_results if k not in ordered_kernels)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for idx, kernel_name in enumerate(ordered_kernels[:2]):
    ax = axes[idx]
    per_class = benchmark_results[kernel_name]['per_class_accuracy']

    colors = [
        'green' if acc >= benchmark_results[kernel_name]['overall_accuracy'] else 'orange'
        for acc in per_class
    ]

    ax.bar(range(10), per_class, color=colors, alpha=0.8, edgecolor='black')
    ax.set_xticks(range(10))
    ax.set_xticklabels([c[:5] for c in cifar10_classes], rotation=45, ha='right')
    ax.set_ylim([0, 100])
    ax.set_ylabel('Accuracy (%)')
    ax.set_title(f'Per-Class Accuracy ({kernel_name})')
    ax.grid(axis='y', alpha=0.25)

# If only 1 kernel, hide the second subplot
if len(ordered_kernels) < 2:
    axes[1].axis('off')

plt.tight_layout()
print("\n✓ Per-class accuracy visualization displayed.")
plt.show()

Execute and Visualize Results for a Selected Kernel

In [ ]:
# 1. Get Data
N_TEST = 1000
ANALYSIS_KERNEL = '3x3'  # Change to '5x5' to inspect the other kernel
tensors_lookup = {
    '3x3': tensors_3x3,
    '5x5': tensors_5x5
}
if ANALYSIS_KERNEL not in tensors_lookup:
    raise ValueError("ANALYSIS_KERNEL must be '3x3' or '5x5'.")

y_true, y_pred = get_predictions(tensors_lookup[ANALYSIS_KERNEL], test_data, n_samples=N_TEST)

# 2. Compute Matrix
conf_matrix = calculate_confusion_matrix(y_true, y_pred)

# 3. Compute Metrics
precisions, recalls, f1s = calculate_metrics(conf_matrix)

# 4. Visualization
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title(f'Confusion Matrix ({ANALYSIS_KERNEL} kernel, N={N_TEST})', fontsize=15)
plt.colorbar()
tick_marks = np.arange(10)
plt.xticks(tick_marks, cifar10_classes, rotation=45)
plt.yticks(tick_marks, cifar10_classes)

thresh = conf_matrix.max() / 2.
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        plt.text(j, i, format(conf_matrix[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if conf_matrix[i, j] > thresh else "black")

plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()

# 5. Print Classification Report Table
print(f"{'Class':<12} | {'Precision':<10} | {'Recall':<10} | {'F1-Score':<10}")
print("-" * 50)

for i, class_name in enumerate(cifar10_classes):
    print(f"{class_name:<12} | {precisions[i]:.4f}     | {recalls[i]:.4f}     | {f1s[i]:.4f}")

print("-" * 50)
print(f"{'Macro Avg':<12} | {np.mean(precisions):.4f}     | {np.mean(recalls):.4f}     | {np.mean(f1s):.4f}")